<font color='lightblue'>Nom Etudiant 1: ELLIETHY Ossama 28712429
<br>
Nom Etudiant 2: HORCHANI Samy 28706765 </font>

# TME 2.2 : Median String


## Partie A : Recheche de pattern (motifs) en permettant des variations

Les motifs que nous cherchons dans les sequences d'ADN peuvent contenir quelques variations ou mutations. Nous allons developpé dans cet partie differents algoritmes pour la recherche de motifs variables. 

1\. Comme dans le TME precedent, nous alons d'abord générer des données atificielles pour pouvoir tester les algorithmes. Faire une fonction pour générer `t` séquences artificielles de taille `n`. Implantez dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement.

In [80]:
import random

nuc = ('A', 'C', 'G', 'T')

k=4 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence

def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n:int, upper=False):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront minuscule, False majuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    for i in range(n):
        sequence = sequence + random.choice(nuc)
    if(upper):
        sequence.lower()
    return sequence

def modifierMotif(motif:str, nbpos:int,  upper=False):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront minuscule, False majuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)

    pos_modifie = []
    for i in range(nbpos):
        random_pos = random.randint(0,len(motifM)-1)

        while(random_pos in pos_modifie):
            random_pos = random.randint(0,len(motifM)-1)
        
        pos_modifie.append(random_pos) #sauvegarde de la position pour pas la modifier 2 fois
        random_nuc = random.choice(nuc)

        while(random_nuc == motif[random_pos]):
            random_nuc = random.choice(nuc)

        if(upper):
            motifM[random_pos] = random_nuc.lower()
        else:
            motifM[random_pos] = random_nuc.upper()
    return "".join(motifM)


print (modifierMotif("acg", 2))


def implantMotifVar(k, v, t, n):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, 
    pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    
    sequences = []
    for i in range(t):
        sequences.append(generateRandomSequence(n-k))
        sequences[i] = insertMotif(sequences[i], modifierMotif(generateRandomSequence(k), v), random.randint(0, n-k-1))
    return sequences


adn = implantMotifVar(k, v, t, n)
print (adn)


aGT
['AGTTTGGGGG', 'TGGGTTTACC', 'CGGTCCCGGC']


2\. Visualisation de motifs. Nous pouvons visualiser les motifs à l'aide des outils de LOGO https://weblogo.berkeley.edu/logo.cgi. Executer votre fonction de generation de motif variable, extraire les motifs et visulise-le à l'aide de webLogo. Utiliser les parametre ci-dessous.


In [81]:
k=8 #taille de motif
v=2 #nb de positions variable dans le motif
t=10 #nb de sequences
n=10 #longuer des sequence

adn = implantMotifVar(k, v, t, n)
print (adn)

['TAATGATACA', 'AGGCCCGATT', 'GGGCATGACA', 'GGACAGCTCT', 'GCCCGTGCAT', 'TGACCACCCT', 'CGCAGATCCA', 'TCTCTCATAG', 'TAGGCCGAGC', 'TCTCCTTCCA']


3\. Implémentez l'algorithme _"Median String Search"_ pour chercher des motifs de taille variable. Tester le sur les jeux de données gnerer precedement. Vous devez éliminer les motifs peu complexe pour éviter les calculs inutiles. Pensez à eliminer tous les motifs ayant une formation diferents de motifs attendues, par exemple, les motifs ayant deux base repeté comme ACACACAC, etc.  

In [82]:
from itertools import product

def kmerList(allKmers):
    return ["".join(nucTuple) for nucTuple in allKmers]



#Enlever les motifs peu complexe


def removeLowComplexe(motifs, m=5):
    motifsClean = []
    suppr=False
    for key in motifs:
        for i in range (len(key)):
            cpt=key.count(key[i])
            if cpt>=m:
                suppr=True
            if(i+3< len(key)):
                if key[i]==key[i+2] and key[i+1]==key[i+3]:
                    suppr=True     
        if not (suppr):
            motifsClean.append(key)
        suppr=False  

    return motifsClean
                
#Genere tous les K-mers de taille K ayant de AAA... à TTT...
allkmers = product(nuc, repeat=k)
kmers = kmerList(allkmers)


print (len(kmers))
kmersValid = removeLowComplexe(kmers, 4)
print (len(kmersValid))
print (kmersValid[0])



65536
29688
AAACCCGG


In [83]:
def hamdist(str1:str, str2:str):
	"""
	Calcul la distance de hamming entre deux chaînes de caractères
	entrée str1: chaîne de caractères
	entrée str2: chaîne de caractères
	sortie distance: distance de hamming
	"""
	diffs = 0

	for i in range(0,len(str1)):
		if (str1[i]!=str2[i]):
			diffs+=1
	return diffs


def totalDistance(motif:str, sequences, k):
	"""
	Calcul la totalDistance
	entrée motif: sequence consensus
	entrée sequences: chaîne de caractères
	entrée k: taille du motif 
	entrée sequences: matrice de dimension txn contenant les séquences
	sortie totalDistance: somme de distance de hamming minimal
	"""
	td = 0
	for i in range(len(sequences)):
		sequence = sequences[i]
		lenSeq = len(sequence)
		distList = [hamdist(motif, sequence[j: j+k]) for j in range(lenSeq-k+1)]
		td += min(distList)
	return td


def MedianStringSearch(allkmers, sequences, k):
	"""
	Implement l'algorithme MedianStringSearch
	entrée allkmers: liste de K-mers valides
	entrée sequences: matrice de dimension txn contenant les séquences
	entrée k: taille du motif 
	sortie bestMotif: le motif que minimise les distances
	sortie bestDistance: la distance minimal
	sortie motifDist: un dictionnaire contenant les motifs et leurs distances
	"""
	bestDistance = 1000
	bestMotif = ''
	motifDist = {}
	for kmer in allkmers:
		distTmp = totalDistance(kmer, sequences, k)
		if distTmp < bestDistance:
			bestMotif, bestDistance = kmer, distTmp
		motifDist[kmer] = distTmp
	return bestMotif, bestDistance, motifDist

bestMotif, bestDistance, motifDist = MedianStringSearch(kmersValid, adn, k)
print(bestMotif, bestDistance)

GACCGATC 34


In [84]:
#Apliquez l'algoritme MedianStringSearch sur les sequences du brin complementaire

def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])




## Partie B : Recherche de motifs variable sur vos données

Chercher sur le fichier "Sequence_by_Peaks_G*.fasta" contiens les regions de peak trouvé par ChipSeq, qui contient probablement le Facteur de Transcription que nous cherchons. Apliquer l'alogorithm Median Sting pour chercher les motifs dans votre fichier "Sequence_by_Peaks_G*. Il faut bien evidement enlever les motifs peu complexe.

In [85]:
import time

k=7
feq=10
top= 50
genome = "Sequence_by_Peaks_1.fasta"



def readFasta(fastaFileName):
    """
    Read a fasta file
    entrée fastaFileName: nom du fichier fasta
    sortie sequences: liste contenant toutes les sequences du fichier
    """
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())

    return sequence

sequences   = readFasta(genome)


In [87]:
allkmers = product(nuc, repeat=k)
kmers = kmerList(allkmers)

kmersValid = removeLowComplexe(kmers, 4)

bestMotif, bestDistance, motifDist = MedianStringSearch(kmersValid, sequences[0:3], k)

print (bestMotif, bestDistance)

AAACTGC 3
